# Day 16: [Reindeer Maze](https://adventofcode.com/2024/day/16)

## Pseudo code
1. Store input map as an array
2. Start from S facing eastward ('>'), save junctions as a dictionary. If a path is blocked, start from the last junction.
3. Calculate points and find the lowest possible points
    - Movement : +1
    - Rotation : +1000

## 1. Input to map array

In [1]:
with open('day16-1.txt','r') as f:
    input_text = [lines.strip() for lines in f]
input_text

['#############################################################################################################################################',
 '#.#.........#...#.....................#...........#...............#...#.................#...........#...#...#.....#.#...........#.....#....E#',
 '#.#.#.#######.#.#.#########.#########.###.#.#####.###.#####.#.###.#.#.#########.#####.#.#####.#.#####.#.#.#.#.###.#.#.#####.#.#.#####.#.###.#',
 '#...#.........#.....#.........#.....#.......#...#.........#.#...#...#...#...#...#.#.....#...#.#.......#...#.#...#.#...#.....#.#.#.....#.#...#',
 '#.###################.###.###.#.###.#####.#####.#####.###.###.#.#######.#.#.#.###.#.#.###.#.###.###########.###.#.#####.###.#.#.#.#.###.#.#.#',
 '#.#.....#.....#...#...#.......#.#.#.#...........#.......#...#...#.....#...#...#.#...#.....#...#.........#...#.#.#...#...#.....#.#.#.#...#...#',
 '#.#.###.#.#####.#.#.#####.###.#.#.#.#.#.#.#####.#.###.#.###.#.#######.#######.#.#.#.#.#.#####.#########.#.#.#.#.###.#.###.

In [2]:
# Store map as an array

import numpy as np
rmap = np.array(input_text)
rmap = np.array([np.array([*r]) for r in rmap], dtype = object)
print(rmap)

[['#' '#' '#' ... '#' '#' '#']
 ['#' '.' '#' ... '.' 'E' '#']
 ['#' '.' '#' ... '#' '.' '#']
 ...
 ['#' '.' '#' ... '#' '#' '#']
 ['#' 'S' '.' ... '.' '.' '#']
 ['#' '#' '#' ... '#' '#' '#']]


## 2. Track movements from S(tart) to E(nd)

In [3]:
# Define variables to keep track of

routes = []                                         # Store successful routes
dir_hist = ['>']                                    # Store past directions
pos_hist = [list(zip(*np.where(rmap == 'S')))[0]]   # Store past positions
junct = []                                          # Store junctions i.e. positions with more than 1 direction

In [4]:
def movement(current_l, next_l):
    '''
    Identify direction to take based on available path (next_l)
    from current path (current_l). Paths already taken are marked as 'X'
    to avoid going backwards
    '''

    if (current_l[1] + 1) == next_l[1]:            # Right
        trace = '>'
    elif (current_l[1] - 1) == next_l[1]:          # Left
        trace = '<'
    elif (current_l[0] - 1) == next_l[0]:          # Up
        trace = '^'
    elif (current_l[0] + 1) == next_l[0]:          # Down
        trace = 'v'
    # rmap[current_l[0],current_l[1]] = 'X'           # Mark the path taken

    return (next_l, trace)

In [5]:
pos = list(zip(*np.where(rmap == 'S')))[0]    # Current position Location of S                                    # Place holder for storing location just before
end = 0 # Condition to end while loop

while ~end :

    junct_count = 0
    temp = []

# Identify possible directions ('.') from the position. If the end (E) is reached, update the list of successful routes

    if (rmap[pos[0]-1,pos[1]] == '.') and ([pos[0]-1,pos[1]] not in pos_hist):    # Up open
        temp.append([pos[0]-1,pos[1]])
        junct_count += 1
    elif rmap[pos[0]-1,pos[1]] == 'E':
        routes.append(dir_hist)

    if (rmap[pos[0]+1,pos[1]] == '.') and ([pos[0]+1,pos[1]] not in pos_hist):    # Down open
        temp.append([pos[0]+1,pos[1]])
        junct_count += 1
    elif rmap[pos[0]+1,pos[1]] == 'E':
        routes.append(dir_hist)

    if (rmap[pos[0],pos[1]-1] == '.') and ([pos[0],pos[1]-1] not in pos_hist):    # Left open
        temp.append([pos[0],pos[1]-1])
        junct_count += 1
    elif rmap[pos[0],pos[1]-1] == 'E':
        routes.append(dir_hist)

    if (rmap[pos[0],pos[1]+1] == '.') and ([pos[0],pos[1]+1] not in pos_hist):    # Right open
        temp.append([pos[0],pos[1]+1])
        junct_count += 1
    elif rmap[pos[0],pos[1]+1] == 'E':
        routes.append(dir_hist)

# How many possible paths are there from the position?

    # Just one way forward
    if junct_count == 1:
        pos, trace = movement(pos,temp[0])
        pos_hist.append(pos)
        dir_hist.append(trace)

    # More than one way, a junction.
    # Save in a list [0: (the location) 1: paths taken until the location, 2: directions (arrows) taken until the location , 3 and onward: unexplored paths]
    elif junct_count > 1:
        temp.insert(0,(pos[0],pos[1]))
        temp.insert(1,pos_hist.copy())
        temp.insert(2,dir_hist.copy())
        pos, trace = movement(pos,temp[-1])
        dir_hist.append(trace)
        pos_hist.append(pos)
        temp.pop()
        junct.append(temp)

    # A dead end, go back to the last junction
    else:
        try:    # Remove already explored junctions
            while (len(junct[-1]) == 3):
                junct.pop()
        except:
            end = 1
            break

    # Every time a path is explored from a junction, remove the path
        pos_hist = junct[-1][1]
        dir_hist = junct[-1][2]
        pos, trace = movement(junct[-1][0],junct[-1][-1])
        dir_hist.append(trace)
        pos_hist.append(pos)
        junct[-1].pop()

## 3. Calculate points and find the lowest possible points

In [6]:
routes

[['>',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '>',
  '>',
  'v',
  'v',
  '>',
  '>',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '>',
  '>',
  'v',
  'v',
  '>',
  '>',
  '^',
  '^',
  '>',
  '>',
  'v',
  'v',
  '>',
  '>',
  'v',
  'v',
  '<',
  '<',
  '<',
  '<',
  'v',
  'v',
  'v',
  'v',
  '>',
  '>',
  '^',
  '^',
  '>',
  '>',
  '>',
  '>',
  '^',
  '^',
  '^',
  '^',
  '^'],
 ['>',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '>',
  '>',
  'v',
  'v',
  '>',
  '>',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '>',
  '>',
  'v',
  'v',
  '>',
  '>',
  '^',
  '^',
  '>',
  '>',
  'v',
  'v',
  '>',
  '>',
  'v',
  'v',
  '<',
  '<',
  '<',
  '<',
  'v',
  'v',
  'v',
  'v',
  '<',
  '<',
  'v',
  'v',
  '<',
  '<',
  'v',
  'v',
  '>',
  '>',
  '>',
  '>',
  '>',
  '>',
  '^',
  '^',
  '>',
  '>',
  '^',
  '^',
  '>',
  '>',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^'],
 ['>',
  '^',
  '^',
  '^',
  '^',
  '^',
  '^

In [7]:
scores = []

for route in routes:
    score = len(route)
    for path in range(len(route)-1):
        if route[path] != route[path+1]:
            score += 1000
    scores.append(score)
scores

[19060,
 25076,
 14040,
 14044,
 17056,
 23072,
 18064,
 20064,
 11048,
 11048,
 12048,
 14048,
 19068,
 25084,
 14048,
 14052,
 17064,
 23080,
 18072,
 20072,
 11056,
 11056,
 12056,
 14056]

In [8]:
min(scores)

11048